<a href="https://colab.research.google.com/github/bhargavrakholiya123/ML-Progress/blob/main/Chatbot_Using_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
uploaded = files.upload()

Saving pandas_notes.pdf to pandas_notes.pdf


In [6]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.6 MB/s eta 0:00:00


In [7]:
import PyPDF2

def extract_text_from_pdf(path):
    text = ""
    with open(path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        for page in reader.pages:
            text += page.extract_text()
    return text

pdf_text = extract_text_from_pdf(list(uploaded.keys())[0])
print(pdf_text[:2000])  # show first part of text


1. Introduction to Pandas
Pandas is an open-source Python library for 
data manipulation and analysis.
You can use Pandas to:
See who bought the most
Add a new column: "Did they tip?"
Save your customer list to a ﬁle
Find out the average number of lemonades bought
And you don’t need to count with ﬁngers — 
Pandas does it with just one line of code!
Install and Import:
pip install pandas
import pandas as pd
pd.__version__
Key Data Structures:
Series = one list
DataFrame = full table
2. Pandas Series
Create Series:
sales = pd.Series([2, 3, 5])
Indexing and Slicing:
sales[1]
sales[0:2]
Attributes:
sales.index
sales.values
sales.dtypeMethods:
sales.head()
sales.tail()
sales.sort_values()
sales.count()
sales.isnull()
Operations:
sales + 1
sales * 2
Missing Values:
sales_with_missing = pd.Series([2, None, 5])
sales_with_missing.isnull()
sales_with_missing.fillna(0)
3. Pandas DataFrame
Create DataFrame:
data = pd.DataFrame({
    'Name': ['Anna', 'Ben'],
    'Lemonades': [2, 3]
})
s1 = pd.Seri

In [8]:
def chunk_text(text, chunk_size=500):
    chunks = []
    words = text.split()
    for i in range(0, len(words), chunk_size):
        chunk = " ".join(words[i:i+chunk_size])
        chunks.append(chunk)
    return chunks

chunks = chunk_text(pdf_text)
len(chunks)


1

In [13]:
from google.colab import userdata
MY_API = userdata.get('GEMINI')
import google.generativeai as genai
genai.configure(api_key=MY_API)
model = genai.GenerativeModel("gemini-2.5-flash")

In [11]:
import numpy as np

def embed_text_list(text_list):
    embeddings = []
    for t in text_list:
        result = genai.embed_content(
            model="models/text-embedding-004",
            content=t
        )
        embeddings.append(result['embedding'])
    return embeddings

chunk_embeddings = embed_text_list(chunks)
len(chunk_embeddings)


1

In [15]:
import numpy as np

def cosine_similarity(a, b):
    a = np.array(a)
    b = np.array(b)
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


In [16]:
def rag_query(question):
    # 1. embed question
    q_embed = genai.embed_content(
        model="models/text-embedding-004",
        content=question
    )['embedding']

    # 2. compute similarity with chunks
    similarities = [cosine_similarity(q_embed, c) for c in chunk_embeddings]

    # 3. get top 3 most relevant chunks
    topk_idx = np.argsort(similarities)[-3:][::-1]
    context = "\n\n".join([chunks[i] for i in topk_idx])

    # 4. ask Gemini with context
    prompt = f"""
    You are a RAG assistant. Use ONLY this context to answer:

    CONTEXT:
    {context}

    QUESTION:
    {question}

    If answer not in context, say "Not found in document".
    """

    response = model.generate_content(prompt)
    return response.text


In [17]:
rag_query("What is the main purpose of this document?")


'The main purpose of this document is to introduce Pandas, which is an open-source Python library for data manipulation and analysis. It explains what Pandas is used for, how to install and import it, and details its key data structures, operations, and various functionalities across different aspects of data handling and visualization.'